In [169]:
%load_ext autoreload
%autoreload 2

import util
import config


project_id = 119867
pool_id = 36425808

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [402]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(12)

In [403]:
import pandas as pd
from collections import Counter

In [170]:
from tqdm.auto import tqdm

In [171]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200", basic_auth=('elastic', 'changeme'))
es_client.info()

ObjectApiResponse({'name': '29617d95b9d1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'axIx5HhRQY6iIouQ_OkZ1A', 'version': {'number': '8.5.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c1310c45fc534583afe2c1c03046491efba2bba2', 'build_date': '2022-11-09T21:02:20.169855900Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [188]:
def save_hashes(attachment_id, doc):
    es_client.index(index="attachment", id=attachment_id, document=doc)

def find_duplicates(doc):
    keys = ['dhash', 'phash', 'md5']
    should = [{"term": {k: doc[k]}} for k in keys]
    query = {'bool': {'should': should}}

    response = es_client.search(query=query, index='attachment')
    return response['hits']

def has_duplicates(doc):
    response = find_duplicates(doc)
    return len(response['hits']) > 0

In [316]:
def get_available_suits(pool_id, last_id=None, status='SUBMITTED'):
    tasks_url = f'api/v1/assignments'

    params = {
        'status': status,
        'pool_id': pool_id,
        'sort': 'id',
        'limit': 100,
    }

    if last_id is not None:
        params['id_gt'] = last_id
    
    suits = []
    ids = set()

    while True:
        resp = util.toloka_get(tasks_url, params).json()

        items = resp['items']
        suits.extend(items)
        ids.update({i['id'] for i in items})

        print(f'added {len(items)}, now have {len(suits)} suits ({len(ids)} unique)')

        if resp['has_more'] == False:
            break

        params['id_gt'] = items[-1]['id']

    return suits, items[-1]['id']

In [404]:
all_suits, last_id = get_available_suits(pool_id, last_id='00022bd050--6378039edb0e8f36d8eea3e5')

added 100, now have 100 suits (100 unique)
added 100, now have 200 suits (200 unique)
added 100, now have 300 suits (300 unique)
added 100, now have 400 suits (400 unique)
added 100, now have 500 suits (500 unique)
added 100, now have 600 suits (600 unique)
added 100, now have 700 suits (700 unique)
added 100, now have 800 suits (800 unique)
added 100, now have 900 suits (900 unique)
added 100, now have 1000 suits (1000 unique)
added 100, now have 1100 suits (1100 unique)
added 100, now have 1200 suits (1200 unique)
added 100, now have 1300 suits (1300 unique)
added 100, now have 1400 suits (1400 unique)
added 100, now have 1500 suits (1500 unique)
added 100, now have 1600 suits (1600 unique)
added 100, now have 1700 suits (1700 unique)
added 100, now have 1800 suits (1800 unique)
added 100, now have 1900 suits (1900 unique)
added 100, now have 2000 suits (2000 unique)
added 100, now have 2100 suits (2100 unique)
added 100, now have 2200 suits (2200 unique)
added 100, now have 2300 sui

In [407]:
last_id

'00022bd050--63789c3b9717e2700065ade9'

In [408]:
image_urls = []

for suite in all_suits:
    for sol in suite['solutions']:
        image_urls.append(sol['output_values']['image'])
        
len(image_urls)

4031

In [409]:
image_hashes = util.map_progress(pool, image_urls, util.download_image_and_calc_hashes)

  0%|          | 0/4031 [00:00<?, ?it/s]

downloading images/eu.76027331-20a7-4e80-9ecc-093930d5be54...
downloading images/eu.86271e0d-b151-4d54-b56c-4fdd78f3b79f...
downloading images/eu.9ee1d792-bfa9-4589-8fdc-3892253df8db...
downloading images/eu.456682e9-cda1-4db0-9b74-446f8ea3e10c...
downloading images/eu.35fd69b6-b478-44f2-b012-01099c7c8b3d...
downloading images/eu.b08129da-08b8-48d9-80cb-3c1a20bd0be8...
downloading images/eu.d5b061d3-5efa-4202-907f-1dd53bc7d06b...
downloading images/eu.01d687d1-aac4-4cd0-9a4a-de308d58a5d2...
downloading images/eu.ee78b74d-b827-40a4-bff8-e9e2084d0b76...
downloading images/eu.a1f5d090-2c3d-4b2d-b512-5d7771d9807e...
downloading images/eu.aff77f78-0cc0-4d59-9b60-0c42f3e3ea26...
downloading images/eu.1acc2bc9-00cb-49e7-b251-4e6fd19ec87f...
downloading images/eu.570cf771-41c6-41d1-85f8-bc1716e65c5b...
downloading images/eu.5879a5f0-0f9f-4790-9fa9-4700319af1e4...
downloading images/eu.3d819efb-596a-4eb8-b776-d4f374e57360...
downloading images/eu.fc815ee3-1ba7-4b6d-b1fe-20bb20d19f38...
download

downloading images/eu.cdc4d6f0-4d99-46bb-9b50-4c3c20334605...
downloading images/eu.c90c0803-1104-45f0-a5e8-57ebdf9c5c0e...
downloading images/eu.7151774b-ff99-4449-8122-b7c8012bbc23...
downloading images/eu.b8b56949-988b-4c19-a9ec-77aabfd22087...
downloading images/eu.9750e305-c8c1-429a-98d0-91b87bcaff78...
downloading images/eu.8a8024aa-c9bd-48b7-b213-de64287642ce...
downloading images/eu.569e5bb1-470b-412f-8a36-b6cddf779acf...
downloading images/eu.c8943ee8-a82c-4f28-b7b1-c9364d0bfa6b...
downloading images/eu.ac36aab0-eb6a-4e88-aff5-b701e426f1f4...
downloading images/eu.38f705bd-9092-46ae-b95d-e644505fbd26...
downloading images/eu.85bb4b47-6fb1-4180-92c4-5d34fec18ccf...
downloading images/eu.ec87d054-7c13-4818-99f5-7d44eda99db0...
downloading images/eu.4f168979-ede0-40f7-9bdc-1d39074ebac5...
downloading images/eu.e319eb90-7a7e-445b-9afb-4bdbe497bdc7...
downloading images/eu.dbc426d7-d009-42be-9bbb-4a2ebc4d77df...
downloading images/eu.639fcccc-68a9-478b-ad02-966aac9d1650...
download

downloading images/eu.61267f03-8964-4e8d-a0b7-c0b2364ecd58...
downloading images/eu.9e31ae25-0296-402c-8e70-23285c4b6e7d...
downloading images/eu.885e41fa-195d-43c0-bcb0-076ed1c2578a...
downloading images/eu.14112615-2ebf-4c31-a3a0-07ef3339708d...
downloading images/eu.60d0e94f-7b61-477a-bbf0-5f1be2345eb4...
downloading images/eu.27d330dd-46a0-4a08-a377-ff4e0dcba858...
downloading images/eu.40e4b158-6e75-43e1-b48f-a9c1154260fc...
downloading images/eu.6480c03c-6b40-4652-91a4-31e9141d8d68...
downloading images/eu.0ed329a3-5879-4fb8-85f3-6988992c1088...
downloading images/eu.36626780-941f-4141-8362-bdfc4aabc55c...
downloading images/eu.4636dce1-aa15-4406-9f4a-e1b034760534...
downloading images/eu.24ef0591-17ba-4737-98a3-abf5a186e96d...
downloading images/eu.d5390a10-5f4b-4be2-a6d4-0ca37c0cd118...
downloading images/eu.15f96d3d-f678-4336-a60c-d2fd7fef2321...
downloading images/eu.ed8ee713-0851-4415-aa5e-d426995714c6...
downloading images/eu.54f34704-c3bb-4772-912c-b6ff0366e651...
download

downloading images/eu.1d3df06e-01fd-4964-b22e-787d9e37dbdf...
downloading images/eu.798ad87e-ef11-456a-b19e-10e129695fe5...
downloading images/eu.75e885cc-fe1c-426e-ac02-2d3294119db3...
downloading images/eu.e207bfe2-f5e2-4ce4-9460-e35fdf2635ce...
downloading images/eu.a15492e8-1071-455d-90e0-12abcb231e2d...
downloading images/eu.07bfdc35-8c31-4970-b400-1f8180120a10...
downloading images/eu.76f9fc8e-74fa-45f6-a033-05f97f13eaec...
downloading images/eu.43a8b5dc-1172-4be3-847a-3ae0e10371e6...
downloading images/eu.c562a7dd-fa9c-4521-9ce4-80712aab582b...
downloading images/eu.1995f499-d54d-4cda-9c98-d69415d8ca32...
downloading images/eu.729e2bde-0de8-42db-9762-52b7915455a8...
downloading images/eu.88dac6f3-6f2b-47e1-acbf-c69fd5caa01f...
downloading images/eu.b33c4603-2eae-45d7-9e49-4b26561c1616...
downloading images/eu.a3cae2db-481f-4375-a077-793ee74f7ad9...
downloading images/eu.7d862512-0088-4065-9745-0f177ed86404...
downloading images/eu.e5d97a1a-c41e-4742-9bd6-251c1f889754...
download

...

downloading images/eu.b602dadb-6564-494f-bc7b-00772d1ce26d...
downloading images/eu.f70469c8-a55f-45ad-9da0-0206b719d505...
downloading images/eu.4f179310-0696-4e7f-bd85-3471fbffbfd1...
downloading images/eu.4cd07157-fea5-450f-ba1e-4ca9d52a1e72...
downloading images/eu.05386bbe-e583-4bb7-b5a3-179ca0fdfbab...
downloading images/eu.0dfca418-a6ed-4d11-9535-00c962774fe2...
downloading images/eu.38b50109-7120-4f86-8f39-ef39682f6368...
downloading images/eu.640c4cc0-3664-4b81-86a3-466d56d01447...
downloading images/eu.8e644e6a-5201-4e2c-b3d6-c8523a9c3917...
downloading images/eu.fc3fdb33-ffbd-410b-b84c-8affe49b28e6...
downloading images/eu.17498d34-4769-4a7b-ae7a-6379940ca507...
downloading images/eu.afb055db-ca8d-4193-afcf-0745c3df986a...
downloading images/eu.c843f185-2fca-4f92-b084-c388b0c9669a...
downloading images/eu.d102c3c8-785d-469a-a9e0-6247405280ae...
downloading images/eu.6c6b7e59-0db9-4021-a725-7074c8a1619a...
downloading images/eu.ecf682e6-e89b-4007-a611-7021de129f91...
download

downloading images/eu.900f276a-b550-4a0d-8c8b-a5e515d35d01...
downloading images/eu.4526e986-406b-437c-b02c-692c420911de...
downloading images/eu.9e5d4750-fb28-4ca1-ab04-60f540157da4...
downloading images/eu.17c20f37-7d63-4d80-a222-8d4c1dd17f5c...
downloading images/eu.752eb289-9b09-43e0-95fa-2e29d241adea...
downloading images/eu.40a4a77f-d6d7-4767-bd64-6adaf68e84ef...
downloading images/eu.61a460a7-0896-4a36-aa62-8912269c3219...
downloading images/eu.10f6ffa5-8cbf-41ec-8e51-77a6a90809a5...
downloading images/eu.227f3d4c-b3e6-4808-a169-5dc349822ab7...
downloading images/eu.0f7d162a-7275-4021-adf0-32f17a575a17...
downloading images/eu.a98f8493-dab0-4c34-a352-66c45e794ed8...
downloading images/eu.2f87322e-b4aa-4b18-9ba3-ee4ba14664bb...
downloading images/eu.875e7ed0-4bd1-40e5-860c-b84a0929714c...
downloading images/eu.1ad21012-192c-4228-9179-6b6d682fd7fd...
downloading images/eu.aed5cc21-1953-48e8-aa5e-621d57d26b0d...
downloading images/eu.186a9559-6f72-4501-8d9a-063abf37bd77...
download

downloading images/eu.e3eebd03-32c5-44ca-9203-98e638faecc4...
downloading images/eu.86bccaff-e390-47af-9d83-5d3ce09feb4b...
downloading images/eu.8d665014-2492-4084-9bdb-08f14dea6416...
downloading images/eu.d7c292e5-f9c9-4655-a7aa-9b53a859abf4...
downloading images/eu.80d6506e-16ab-4f43-a601-42529bc44cc3...
downloading images/eu.9153a1e1-f820-42f0-8627-ba2f5ec037e2...
downloading images/eu.7986f793-30fe-4def-9e9d-70ebfdac8c80...
downloading images/eu.b0f7d2d6-d922-449e-9b47-4f3a29cbd357...
downloading images/eu.a5aecda1-3427-4c66-846a-0f106286be13...
downloading images/eu.1870ecdb-24b0-402e-99f3-e940f7e408e6...
downloading images/eu.2b8d7a00-f1a1-46be-a5ce-e802bf17ed9a...
downloading images/eu.8c0bec87-ea0f-48e1-b582-433101011129...
downloading images/eu.99851c9e-2811-49a7-bf3a-9352ea792197...
downloading images/eu.79f55774-bee9-45fb-9043-b69adcccc928...
downloading images/eu.de2b1d24-3820-42e7-b48d-46f1db9826ff...
downloading images/eu.db446d56-b840-4672-b5fb-bea1d8ac4342...
download

downloading images/eu.e200baae-c271-4fda-8a8d-5e1d1b382c53...
downloading images/eu.a5f442ed-ebcd-4bfe-8bff-5ad72003fdc7...
downloading images/eu.287329ed-f97a-46a9-998f-56471920423b...
downloading images/eu.b05e2e3c-38a0-464f-8e35-8219256a33a0...
downloading images/eu.9a445619-d807-47ea-9dfd-ec5905f33e02...
downloading images/eu.64dd5d46-1cf8-4264-8943-3ab78ae77cf1...
downloading images/eu.1b94e84a-22e5-4147-8a58-02908bf2899a...
downloading images/eu.b0e6aa11-8a5c-4f0e-8b2a-828337af6735...
downloading images/eu.37435d2f-abb8-47cb-b14a-2de83270165b...
downloading images/eu.2be3bea0-84ae-4728-9219-5f7db3a1e649...
downloading images/eu.7572a5a6-6d72-4003-8303-be2b0775760a...
downloading images/eu.143ead15-86fb-45e5-b61a-b6496965723f...
downloading images/eu.e3a0eba3-dfca-463c-a990-49fbfe4f58c6...
downloading images/eu.d2b2abde-ab57-415d-9b1c-88f83e855c7c...
downloading images/eu.6d1b3d98-3d28-409e-b45d-1eb656bea320...
downloading images/eu.7dde00f2-e8e4-4126-9def-7016f9069fe4...
download

downloading images/eu.90013b3b-da83-4542-8f5b-fc47c721191f...
downloading images/eu.7fa8d831-4045-4a70-950d-e005265abac0...
downloading images/eu.eed9c69c-dbde-4849-ac38-cddeed9674a0...
downloading images/eu.14c77a95-d9d7-439a-8a0e-edf843a47899...
downloading images/eu.5ca1914d-900a-4987-9abc-d5d2bf511a4f...
downloading images/eu.f683ba89-a4f0-4b28-b82b-f9b672579b71...
downloading images/eu.5ad34c52-aa6d-4e03-ba71-2da59f6f9013...
downloading images/eu.1a05a00c-e0e3-47fc-a61e-6376c2b80122...
downloading images/eu.828345ea-ea11-47b4-8c88-c423e762637f...
downloading images/eu.1757424e-12d2-496d-92c9-566c7f1921c0...
downloading images/eu.692ca6ce-c390-4ce6-9eb2-0c57083ff515...
downloading images/eu.3e545a38-4e10-4b2b-a289-069ac030fafb...
downloading images/eu.c242af40-a7cf-4688-9aac-53cc2def0242...
downloading images/eu.b6562bc1-fa5b-4f49-bcf3-d1d8024cc456...
downloading images/eu.129046d9-296b-41c9-aefa-8dcefe704db8...
downloading images/eu.42aed020-6f6c-450f-98bb-729ed3f96534...
download

In [347]:
len(image_hashes)

4115

In [410]:
image_hashes = dict(image_hashes)

In [411]:
processed = set()
to_check = []

In [412]:
for suite in tqdm(all_suits):
    suite_id = suite['id']
    if suite_id in processed:
        print(f'aready processed {suite_id}')
        continue

    task = suite['tasks'][0]
    solution = suite['solutions'][0]

    output_values = solution['output_values']

    output_class = output_values['class']

    if output_class == 'other':
        util.reject(suite_id, 'wrong item type')
        processed.add(suite_id)
        continue

    image = output_values['image']
    hashes = image_hashes[image] #util.download_image_and_calc_hashes(image)
    
    if hashes is None:
        util.reject(suite_id, 'problems with image')
        processed.add(suite_id)
        continue

    if has_duplicates(hashes):
        util.reject(suite_id, 'duplicate')
        processed.add(suite_id)
        continue

    save_hashes(image, hashes)

    image_url = util.make_upload_thumbnail(pool_id, image)
    
    to_check.append({
        'image_url': image_url,
        'worker_id': suite['user_id'],
        'assignment_id': suite_id,
        'attachment_id': image
    })
    
    processed.add(suite_id)

  0%|          | 0/4031 [00:00<?, ?it/s]

saved a thumbnail to thumbnails/eu.76027331-20a7-4e80-9ecc-093930d5be54.jpg
saved a thumbnail to thumbnails/eu.86271e0d-b151-4d54-b56c-4fdd78f3b79f.jpg
saved a thumbnail to thumbnails/eu.9ee1d792-bfa9-4589-8fdc-3892253df8db.jpg
saved a thumbnail to thumbnails/eu.456682e9-cda1-4db0-9b74-446f8ea3e10c.jpg
saved a thumbnail to thumbnails/eu.35fd69b6-b478-44f2-b012-01099c7c8b3d.jpg
00022bd050--637803ba441f157a2a5499f7 REJECTED wrong item type
saved a thumbnail to thumbnails/eu.d5b061d3-5efa-4202-907f-1dd53bc7d06b.jpg
saved a thumbnail to thumbnails/eu.01d687d1-aac4-4cd0-9a4a-de308d58a5d2.jpg
saved a thumbnail to thumbnails/eu.ee78b74d-b827-40a4-bff8-e9e2084d0b76.jpg
saved a thumbnail to thumbnails/eu.a1f5d090-2c3d-4b2d-b512-5d7771d9807e.jpg
saved a thumbnail to thumbnails/eu.aff77f78-0cc0-4d59-9b60-0c42f3e3ea26.jpg
saved a thumbnail to thumbnails/eu.1acc2bc9-00cb-49e7-b251-4e6fd19ec87f.jpg
saved a thumbnail to thumbnails/eu.570cf771-41c6-41d1-85f8-bc1716e65c5b.jpg
saved a thumbnail to thumb

saved a thumbnail to thumbnails/eu.d2c38423-ab5f-4c07-8360-7be609729db3.jpg
saved a thumbnail to thumbnails/eu.b6e7991a-dc93-4fa6-b808-9a2a2e4ee0e1.jpg
saved a thumbnail to thumbnails/eu.b09457e5-1dc9-4096-ae9a-6db77b291dcd.jpg
saved a thumbnail to thumbnails/eu.81614eb6-7ad5-4778-90f4-e5c0d9f8cdfe.jpg
saved a thumbnail to thumbnails/eu.66300bb7-6027-407c-a99a-93c43ca9706e.jpg
saved a thumbnail to thumbnails/eu.3971844b-cde7-4bbf-90da-1f13e2c32ad0.jpg
saved a thumbnail to thumbnails/eu.3e175df0-940e-496a-baf3-ac6ec6c2e6ba.jpg
saved a thumbnail to thumbnails/eu.386155a9-c9c8-4f71-a22c-71124e83e435.jpg
saved a thumbnail to thumbnails/eu.621f13b2-12d8-49dd-9c73-58b30d371a66.jpg
saved a thumbnail to thumbnails/eu.5cda040c-91b4-4fc2-b78a-e88e2b665b12.jpg
00022bd050--637805fe56a5a54ced8151de REJECTED wrong item type
saved a thumbnail to thumbnails/eu.cbe08572-0eb5-4650-872f-af79a999a309.jpg
00022bd050--637806017545ab02b68d611d REJECTED wrong item type
saved a thumbnail to thumbnails/eu.7efee

saved a thumbnail to thumbnails/eu.66b06f09-cadb-40d5-a37d-4768b0711e09.jpg
saved a thumbnail to thumbnails/eu.f035afa8-40e5-4ada-be91-fd74de5a651c.jpg
00022bd050--637807d7801461320b68c0aa REJECTED wrong item type
00022bd050--637807de7adbfb533edb9382 REJECTED duplicate
saved a thumbnail to thumbnails/eu.b32e48b9-4905-427e-8e97-d70dbc2eaff2.jpg
saved a thumbnail to thumbnails/eu.64843fa3-d1cd-4af5-b932-71ff521ec0a8.jpg
saved a thumbnail to thumbnails/eu.99946e4a-b4e9-4f18-9f64-39d648edfcd3.jpg
saved a thumbnail to thumbnails/eu.15ffef2d-42a2-46b4-ac6f-73505f55903a.jpg
00022bd050--637807f6b1ad940367010aa1 REJECTED wrong item type
saved a thumbnail to thumbnails/eu.2773db9d-8dc1-4115-aee6-4be3f0377b77.jpg
saved a thumbnail to thumbnails/eu.d71d58d2-ba88-44ae-8b18-c38016ae1ca8.jpg
saved a thumbnail to thumbnails/eu.d9360bf9-60d5-4d87-b9f2-d73ed40269b5.jpg
00022bd050--6378080738b93276f2f3c4ed REJECTED wrong item type
saved a thumbnail to thumbnails/eu.ed0bcd44-c581-415f-a8ef-51a17a4b0da8.jp

saved a thumbnail to thumbnails/eu.b7793f76-2fad-478b-984f-281e71c4fad7.jpg
saved a thumbnail to thumbnails/eu.a842bb46-438a-48b2-b5b2-0f0512eda73d.jpg
saved a thumbnail to thumbnails/eu.1826037b-94fb-4fae-9ce5-fe1590b21c3e.jpg
saved a thumbnail to thumbnails/eu.fd0c1f6f-2844-42f2-a080-ac33d9272fc9.jpg
saved a thumbnail to thumbnails/eu.19b57a3d-5ede-47c2-83ec-ccb9930803de.jpg
saved a thumbnail to thumbnails/eu.14059b18-e710-473a-8450-1e2fbbc464b8.jpg
saved a thumbnail to thumbnails/eu.fac742da-4ab8-411f-a9cb-931ebb8206aa.jpg
saved a thumbnail to thumbnails/eu.a32e9e5e-3993-44d0-9323-5c038922f825.jpg
saved a thumbnail to thumbnails/eu.c71747b0-ca33-44c9-8e08-141df73213b5.jpg
saved a thumbnail to thumbnails/eu.1824d5fd-4cf8-42c7-948a-107dd0afe341.jpg
saved a thumbnail to thumbnails/eu.7718b413-b5a1-4bfb-8daf-4320c0b792bb.jpg
saved a thumbnail to thumbnails/eu.b866ac1a-847a-43cf-a5c9-a9faebbaee8c.jpg
saved a thumbnail to thumbnails/eu.9a954202-be8e-4384-82f1-6b21e03532bb.jpg
saved a thum

...

saved a thumbnail to thumbnails/eu.45c3ebf0-e69c-4224-9951-93aa4d357012.jpg
00022bd050--637890d81455e654a92dabfc REJECTED wrong item type
saved a thumbnail to thumbnails/eu.4a0e2ce5-ee26-4100-8a49-d9205abeec55.jpg
saved a thumbnail to thumbnails/eu.a2654dea-137a-4902-b58f-7ca973e342a7.jpg
saved a thumbnail to thumbnails/eu.b1b8e54e-0484-4fc2-b542-10f2bcb7159f.jpg
saved a thumbnail to thumbnails/eu.5acc4eb2-6f95-4cc2-b1ea-be065348e6e9.jpg
saved a thumbnail to thumbnails/eu.56e3d260-55c8-4e96-b64e-5ed0acd76206.jpg
saved a thumbnail to thumbnails/eu.ee3ff2b9-123b-436b-9136-eee68dc42d6f.jpg
saved a thumbnail to thumbnails/eu.c0be4634-82f4-4fda-bcd0-d36186cd10e2.jpg
saved a thumbnail to thumbnails/eu.ef7e4f03-04a4-4653-9722-5039a4e9496b.jpg
saved a thumbnail to thumbnails/eu.3a61cf3d-45cf-4821-ad0a-c44c846c6cb8.jpg
saved a thumbnail to thumbnails/eu.7e05f937-c51d-438a-89c6-40f0a83c384b.jpg
saved a thumbnail to thumbnails/eu.b14213bc-b106-4255-8e51-16b7920d85b9.jpg
00022bd050--637890e72e3a4b

saved a thumbnail to thumbnails/eu.e4c7e281-34cd-4746-921e-5aea95344233.jpg
saved a thumbnail to thumbnails/eu.05caf668-e205-4cbe-9903-a8da6f458383.jpg
saved a thumbnail to thumbnails/eu.8c2afc37-de68-443d-9b52-4bd25a7b09f3.jpg
00022bd050--637891828408eb4b99ac59bf REJECTED wrong item type
saved a thumbnail to thumbnails/eu.6be0d831-2921-4f68-a538-d1360e832337.jpg
saved a thumbnail to thumbnails/eu.6e70f7b7-6bb5-47a4-a527-c00c81371075.jpg
saved a thumbnail to thumbnails/eu.fb98d26a-92d8-478d-901a-1e82586a86fd.jpg
saved a thumbnail to thumbnails/eu.13e892ea-763e-4470-828f-b0101256d259.jpg
saved a thumbnail to thumbnails/eu.79c2bccf-26eb-46d1-a1ed-7dafcfca98a7.jpg
saved a thumbnail to thumbnails/eu.016e9602-e097-45da-b808-a4cff2014de6.jpg
saved a thumbnail to thumbnails/eu.6fa64a86-e58d-45c0-9b69-2c4cadcde012.jpg
saved a thumbnail to thumbnails/eu.98575bdf-9b61-4095-9437-33f461797645.jpg
saved a thumbnail to thumbnails/eu.75d6d5fb-eece-46b2-ac00-25d6305ddf41.jpg
saved a thumbnail to thumb

saved a thumbnail to thumbnails/eu.86bccaff-e390-47af-9d83-5d3ce09feb4b.jpg
saved a thumbnail to thumbnails/eu.8d665014-2492-4084-9bdb-08f14dea6416.jpg
00022bd050--6378922185bacb70b67af6aa REJECTED wrong item type
saved a thumbnail to thumbnails/eu.80d6506e-16ab-4f43-a601-42529bc44cc3.jpg
00022bd050--637892255a4d4b776b231f1b REJECTED wrong item type
00022bd050--6378922647042e6e2c6217ff REJECTED wrong item type
saved a thumbnail to thumbnails/eu.b0f7d2d6-d922-449e-9b47-4f3a29cbd357.jpg
00022bd050--63789229aabecc6bd17fa2d2 REJECTED wrong item type
saved a thumbnail to thumbnails/eu.1870ecdb-24b0-402e-99f3-e940f7e408e6.jpg
saved a thumbnail to thumbnails/eu.2b8d7a00-f1a1-46be-a5ce-e802bf17ed9a.jpg
saved a thumbnail to thumbnails/eu.8c0bec87-ea0f-48e1-b582-433101011129.jpg
saved a thumbnail to thumbnails/eu.99851c9e-2811-49a7-bf3a-9352ea792197.jpg
saved a thumbnail to thumbnails/eu.de2b1d24-3820-42e7-b48d-46f1db9826ff.jpg
saved a thumbnail to thumbnails/eu.79f55774-bee9-45fb-9043-b69adcccc

00022bd050--637892b9801461320b6c37a0 REJECTED duplicate
saved a thumbnail to thumbnails/eu.e0671b85-0e38-4d9b-8fed-5adb78539f36.jpg
saved a thumbnail to thumbnails/eu.efba7655-e45b-4d78-9796-bb3e3131d69f.jpg
saved a thumbnail to thumbnails/eu.f7da9355-10bc-498c-8d79-14f3925262af.jpg
saved a thumbnail to thumbnails/eu.c6876942-c6e9-45cf-a5bb-b970b4df05ae.jpg
saved a thumbnail to thumbnails/eu.a497f4d2-18f7-43b6-b0ef-f90e38c696d3.jpg
saved a thumbnail to thumbnails/eu.e191582b-7fc1-433a-92fa-7bedf5bc7ba1.jpg
saved a thumbnail to thumbnails/eu.f960035b-ca68-4c7a-b33b-f8abdf4f8c4d.jpg
saved a thumbnail to thumbnails/eu.c1ef1fda-facd-402b-b42d-0061a79632d7.jpg
saved a thumbnail to thumbnails/eu.d222bc4a-0233-41a8-8c19-8ab323a3a7c0.jpg
saved a thumbnail to thumbnails/eu.cbc96a05-a42a-469b-8f1e-619a2f389051.jpg
00022bd050--637892c97545ab02b6910509 REJECTED wrong item type
00022bd050--637892cc47042e6e2c621e9b REJECTED wrong item type
saved a thumbnail to thumbnails/eu.31e90d79-fe35-4e40-bc7c-6

saved a thumbnail to thumbnails/eu.d9b498b2-aebc-455f-bca8-23648885bb10.jpg
saved a thumbnail to thumbnails/eu.2bb9382c-41f7-4c74-a11c-505b3c9e7f33.jpg
saved a thumbnail to thumbnails/eu.15b627ce-9497-4bf9-98ed-83a7056d9f3d.jpg
saved a thumbnail to thumbnails/eu.f3415b7b-ec30-4004-aa37-150946fc1b23.jpg
00022bd050--6378935185bacb70b67b036e REJECTED wrong item type
00022bd050--63789352148cbf49f56926d7 REJECTED wrong item type
saved a thumbnail to thumbnails/eu.dab66c46-df80-414a-95ce-c54088579579.jpg
00022bd050--63789357659b10691097a9ca REJECTED wrong item type
saved a thumbnail to thumbnails/eu.f25823d4-c2b1-4f40-aeff-672f4040796d.jpg
saved a thumbnail to thumbnails/eu.7f76978b-0913-4ab4-aba4-0d9febbe5134.jpg
saved a thumbnail to thumbnails/eu.2ed21285-a97f-445c-874a-ccfd3d051855.jpg
saved a thumbnail to thumbnails/eu.39017497-5feb-45d8-bf05-496821f31d95.jpg
saved a thumbnail to thumbnails/eu.877eeacd-48c0-4080-ab84-720b40025386.jpg
saved a thumbnail to thumbnails/eu.c8ad356c-4d63-4c84-

## Now let's upload this to Toloka!

In [418]:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result

In [419]:
pool_verify_id = 36426484
pool_upload_url = 'api/v1/task-suites?async_mode=false&open_pool=true'

In [415]:
batches = make_batches(to_check, 10)

In [416]:
len(batches)

312

In [417]:
for batch in batches:
    tasks = [{'input_values': d} for d in batch]

    data = {
        'pool_id': pool_verify_id,
        'tasks': tasks,
        "overlap": 5,
    }

    resp = util.toloka_post(pool_upload_url, data)
    print(resp)
    print(resp.json())

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '8d665a0f-e510-4e11-9997-4434cc821220', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.76027331-20a7-4e80-9ecc-093930d5be54.jpg', 'worker_id': '885bcd7b87604acc452308d2e72c6e6b', 'assignment_id': '00022bd050--637803aa1455e654a9295ba9', 'attachment_id': 'eu.76027331-20a7-4e80-9ecc-093930d5be54'}}, {'id': 'e492fbe0-c524-4a0a-8d73-f69810d54a50', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.86271e0d-b151-4d54-b56c-4fdd78f3b79f.jpg', 'worker_id': 'ad21fd05471cdb3f2c3956f5d0a8ee48', 'assignment_id': '00022bd050--637803b19717e27000619b08', 'attachment_id': 'eu.86271e0d-b151-4d54-b56c-4fdd78f3b79f'}}, {'id': '97881489-8246-448e-a6c0-783d8916dee9', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.9ee1d792-bfa9-4589-8fdc-3892253df8db.jpg', 'worker_id': '74abf

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '2b3a3738-2931-4b49-9233-35c251e8cdbc', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.23d55030-03b8-4b51-b5d3-c016c9715145.jpg', 'worker_id': '9e67ace7eb742d99a08ce47f2be0b99e', 'assignment_id': '00022bd050--637804652e3a4b54f619b3e0', 'attachment_id': 'eu.23d55030-03b8-4b51-b5d3-c016c9715145'}}, {'id': 'ee21b06b-bcb8-432b-b78a-8eed588b7bc6', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.20bc43f9-f404-4bcc-8332-de9258f1423c.jpg', 'worker_id': 'ad21fd05471cdb3f2c3956f5d0a8ee48', 'assignment_id': '00022bd050--63780469b88cb76558d0a3a7', 'attachment_id': 'eu.20bc43f9-f404-4bcc-8332-de9258f1423c'}}, {'id': '8e2188b0-2e4c-424e-a295-1e0266e95430', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.85beb94c-60f8-4533-b9df-96591ed40482.jpg', 'worker_id': 'b04da

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '08b4cb43-4c9e-428b-ae2f-ca2c082168ef', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.0ba97fdd-d8e6-4ad5-9feb-e99dc5010e0e.jpg', 'worker_id': 'a2fd5e7e64dd808fbf0cf1bb757edf18', 'assignment_id': '00022bd050--637804fe8d6def0568e3ddce', 'attachment_id': 'eu.0ba97fdd-d8e6-4ad5-9feb-e99dc5010e0e'}}, {'id': '17e4c6b1-313f-42e5-b880-1d7afffbec65', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c99ad0ab-eac1-418d-b156-e54baa348985.jpg', 'worker_id': 'ad21fd05471cdb3f2c3956f5d0a8ee48', 'assignment_id': '00022bd050--63780506f63f4544852d4920', 'attachment_id': 'eu.c99ad0ab-eac1-418d-b156-e54baa348985'}}, {'id': 'fb93690e-97e6-4d05-a1e3-93d2d700274d', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.77fb21fd-f216-4761-bfa5-224e53c988f0.jpg', 'worker_id': 'ad21f

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': 'e61043b3-6ce7-4f9a-bbac-1cd86f08aaba', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.496b0144-a59b-4d5f-9df3-9b5e3a0b229c.jpg', 'worker_id': '4dbf55105a70bc5ffd319ef60aaa5d2b', 'assignment_id': '00022bd050--637805a3659b1069109256f7', 'attachment_id': 'eu.496b0144-a59b-4d5f-9df3-9b5e3a0b229c'}}, {'id': 'a1689e14-93f6-4899-b03c-5860adbd3ef3', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.0f1d2af8-3059-4fe2-bc10-efa1e13f7a33.jpg', 'worker_id': 'b2da3d3da6fcb42373117e7b1be57886', 'assignment_id': '00022bd050--637805a5c9e3e243ce94b1b4', 'attachment_id': 'eu.0f1d2af8-3059-4fe2-bc10-efa1e13f7a33'}}, {'id': '1aacdfeb-4bdd-43a4-b57b-05300cc43887', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.298eb0dc-1de7-4efa-a892-f7df7d5d4a11.jpg', 'worker_id': 'ad21f

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '5a9aec12-52c8-48c2-8cd5-89057d33ebeb', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.66de5e26-fda6-4b9d-b48f-cf3a044bfaa5.jpg', 'worker_id': 'fdd6f72b9f71d832cecf2a616b58af97', 'assignment_id': '00022bd050--6378062c7545ab02b68d626c', 'attachment_id': 'eu.66de5e26-fda6-4b9d-b48f-cf3a044bfaa5'}}, {'id': '97248d93-a0be-4068-bd32-dd6f818b84c0', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.cdc4d6f0-4d99-46bb-9b50-4c3c20334605.jpg', 'worker_id': 'ad21fd05471cdb3f2c3956f5d0a8ee48', 'assignment_id': '00022bd050--6378062cb04cf4432651f3c8', 'attachment_id': 'eu.cdc4d6f0-4d99-46bb-9b50-4c3c20334605'}}, {'id': 'f3424815-d015-47f5-88d9-0c639a9ad48e', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c90c0803-1104-45f0-a5e8-57ebdf9c5c0e.jpg', 'worker_id': 'f413e

...

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '19a03b78-f81e-4ef1-886e-6ae4ac43e898', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.287329ed-f97a-46a9-998f-56471920423b.jpg', 'worker_id': '3a6588ed3c7efdbc6aeb732966fdfad2', 'assignment_id': '00022bd050--637892d98408eb4b99ac6996', 'attachment_id': 'eu.287329ed-f97a-46a9-998f-56471920423b'}}, {'id': '5909d31f-30ba-4b75-9f36-6905aca4170b', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.b05e2e3c-38a0-464f-8e35-8219256a33a0.jpg', 'worker_id': '95e8a59206dfbf0e6df02f518c62b515', 'assignment_id': '00022bd050--637892d9ee7d356a087d00e6', 'attachment_id': 'eu.b05e2e3c-38a0-464f-8e35-8219256a33a0'}}, {'id': '492cac3f-3b91-406a-a076-31f2ccb3c517', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.9a445619-d807-47ea-9dfd-ec5905f33e02.jpg', 'worker_id': '29241

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': 'c18d586d-ab04-4b24-8767-83171418c9b9', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.af0bf56b-1f46-45cc-838d-3d3c63b2218d.jpg', 'worker_id': '573e0047173faab18e91870dbcb2c4ba', 'assignment_id': '00022bd050--63789303d23d9327ec01403b', 'attachment_id': 'eu.af0bf56b-1f46-45cc-838d-3d3c63b2218d'}}, {'id': '02a310b7-afc2-45b9-8f06-a5ac83a51c44', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.02af49f1-5322-4b79-9bf8-e19ee85005d8.jpg', 'worker_id': '3a6588ed3c7efdbc6aeb732966fdfad2', 'assignment_id': '00022bd050--6378930419c2d72a00b91924', 'attachment_id': 'eu.02af49f1-5322-4b79-9bf8-e19ee85005d8'}}, {'id': '847a17b6-e732-44a8-802a-0a57d3651c47', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.f51ba2db-e868-460a-b641-ca0b083f1ff9.jpg', 'worker_id': '29241

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': 'b6082b77-6905-427e-bf77-9ee4b785685d', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.9ce886bb-d2a3-4726-aa25-c6f481e64889.jpg', 'worker_id': '674fbc58ef753c7d371d2fd4f7be6f48', 'assignment_id': '00022bd050--637893368ab0c64deb784afc', 'attachment_id': 'eu.9ce886bb-d2a3-4726-aa25-c6f481e64889'}}, {'id': '6f9a1cb0-28ae-4a48-b020-cf325bdaeaf6', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.2595a4ea-9220-45c2-9df6-64ad64dd83de.jpg', 'worker_id': 'd2b4042ec6d8505a41b809e64d5adb7c', 'assignment_id': '00022bd050--63789338bb79c0321f84e1e7', 'attachment_id': 'eu.2595a4ea-9220-45c2-9df6-64ad64dd83de'}}, {'id': 'd2849553-9703-40eb-9bad-46fcdc6dd907', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.ecb30fc2-1bef-497a-9c12-1acb529a8ea6.jpg', 'worker_id': 'a46bb

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '5a5cf568-3e01-458c-b946-d3e33e1d8575', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c8ad356c-4d63-4c84-a1d5-021d5b6cc576.jpg', 'worker_id': '087d2ed749948ee0b36b6c8bd6931133', 'assignment_id': '00022bd050--6378935ebdfa5861cd025573', 'attachment_id': 'eu.c8ad356c-4d63-4c84-a1d5-021d5b6cc576'}}, {'id': '22fbec03-630e-4e47-bf84-349b4f8a9d90', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.9901f28b-057e-4269-9465-1d92376e6bdb.jpg', 'worker_id': 'bcd16c1559831c654671a0c798ffbd49', 'assignment_id': '00022bd050--63789360d23d9327ec014359', 'attachment_id': 'eu.9901f28b-057e-4269-9465-1d92376e6bdb'}}, {'id': 'c04e7efa-3adb-4da2-9a59-5d0977d119d4', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.a1f4c005-e1d8-4bd8-b0ac-e78d95880f17.jpg', 'worker_id': 'fa77c

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': 'fc0f3017-917b-4267-b0aa-27cd12c0fbe2', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.eed9c69c-dbde-4849-ac38-cddeed9674a0.jpg', 'worker_id': '3d7cf36a24fe958404d5953fa0d1ad0b', 'assignment_id': '00022bd050--6378948f5a4d4b776b23387d', 'attachment_id': 'eu.eed9c69c-dbde-4849-ac38-cddeed9674a0'}}, {'id': '4c2a2c4b-20a4-4df6-9e44-40e45e5c2ef8', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.5ca1914d-900a-4987-9abc-d5d2bf511a4f.jpg', 'worker_id': '54d9c4b2dffa7d3fdb34d28341cb80ee', 'assignment_id': '00022bd050--637894a456a5a54ced854fe2', 'attachment_id': 'eu.5ca1914d-900a-4987-9abc-d5d2bf511a4f'}}, {'id': 'e20dd673-8c58-4e05-991a-7afb50568783', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.f683ba89-a4f0-4b28-b82b-f9b672579b71.jpg', 'worker_id': '29e29

## Accept/reject the original images

In [420]:
pool_verify_id = 36426484

In [421]:
verify_suits[-1]['tasks'][-1]['id']

'8ddde2c9-6445-44ff-85e6-6cbafc374586'

In [422]:
verify_suits[-1]['id']

'00022bd2f4--6378a6f6ff977c7c90546c25'

In [423]:
verify_suits, last_id = get_available_suits(
    pool_verify_id,
    last_id='00022bd2f4--6378a6f6ff977c7c90546c25',
    status='ACCEPTED'
)

added 100, now have 100 suits (100 unique)
added 100, now have 200 suits (200 unique)
added 100, now have 300 suits (300 unique)
added 100, now have 400 suits (400 unique)
added 100, now have 500 suits (500 unique)
added 100, now have 600 suits (600 unique)
added 100, now have 700 suits (700 unique)
added 100, now have 800 suits (800 unique)
added 100, now have 900 suits (900 unique)
added 100, now have 1000 suits (1000 unique)
added 100, now have 1100 suits (1100 unique)
added 100, now have 1200 suits (1200 unique)
added 100, now have 1300 suits (1300 unique)
added 100, now have 1400 suits (1400 unique)
added 100, now have 1500 suits (1500 unique)
added 60, now have 1560 suits (1560 unique)


In [424]:
last_id

'00022bd2f4--63796a1785bacb70b68343aa'

In [425]:
data = []

for suite in verify_suits:
    for task, solution in zip(suite['tasks'], suite['solutions']):
        input_values = task['input_values']
        output_values = solution['output_values']
    
        row = input_values.copy()
        row.update(output_values)
        row['verifier'] = suite['user_id']
        data.append(row)

In [426]:
len(data)

15590

In [427]:
df = pd.DataFrame(data)

In [428]:
df.verdict.value_counts()

yes    15590
Name: verdict, dtype: int64

In [429]:
gb = df.groupby('assignment_id')

In [430]:
all_results = []

for assignment_id, df_group in gb:
    classes = Counter(df_group['class'])
    verdicts = Counter(df_group['verdict'])
    
    most_common_verdict, _ = verdicts.most_common(1)[0]
    most_common_class, num_votes = classes.most_common(1)[0]

    agreement = num_votes / len(df_group)

    record = {
        'assignment_id': assignment_id,
        'agreement': agreement,
        'image': df_group.iloc[0].image_url,
        'image_id': df_group.iloc[0].attachment_id,
        'most_common_class': most_common_class,
        'most_common_verdict': most_common_verdict,
        'verdicts': verdicts,
        'classes': classes,
    }
    
    all_results.append(record)

In [431]:
df_agreements = pd.DataFrame(all_results)

In [432]:
good = (df_agreements.most_common_verdict == 'yes') & \
    (df_agreements.most_common_class != 'other') & \
    (df_agreements.agreement >= 0.34)

In [433]:
sum(good) / len(good)

0.9300833867864016

In [435]:
df_agreements['final_verdict'] = 'ACCEPT'
df_agreements.loc[~good, 'final_verdict'] = 'REJECT'

In [436]:
df_agreements['comment'] = 'good job'
df_agreements.loc[~good, 'comment'] = "doesn't satisfy the requrements"

In [437]:
def process_agreements(row):
    assignment_id = row['assignment_id']
    verdict = row['final_verdict']
    comment = row['comment']

    if verdict == 'ACCEPT':
        util.accept(assignment_id)
    else:
        util.reject(assignment_id, comment)

In [438]:
rows = df_agreements.to_dict(orient='records')

In [441]:
_ = util.map_progress(pool, rows, process_agreements)

  0%|          | 0/3118 [00:00<?, ?it/s]

00022bd050--637803b4b88cb76558d09e6b ACCEPTED good job
00022bd050--637803ba8408eb4b99a80a5d ACCEPTED good job
00022bd050--637803bb47f33832e65248c8 ACCEPTED good job
00022bd050--637803b7d3a4cf61019bf630 ACCEPTED good job
00022bd050--637803aa1455e654a9295ba9 ACCEPTED good job
00022bd050--637803b88408eb4b99a80a50 ACCEPTED good job
00022bd050--637803b19717e27000619b08 ACCEPTED good job
00022bd050--637803dab04cf4432651d39f ACCEPTED good job
00022bd050--63780411389626777f7894e2 ACCEPTED good job
00022bd050--6378040f38b93276f2f3aa1c ACCEPTED good job
00022bd050--637804102e3a4b54f619b168 REJECTED doesn't satisfy the requrements
00022bd050--637804038d6def0568e3d596 ACCEPTED good job
00022bd050--637803fd441f157a2a549bcb ACCEPTED good job
00022bd050--637803e9148cbf49f565490a ACCEPTED good job
00022bd050--637804011d6dd41f3b9daa41 ACCEPTED good job
00022bd050--63780411441f157a2a549c12 ACCEPTED good job
00022bd050--637804188cae8c05c8279e74 ACCEPTED good job
00022bd050--637803dcaabecc6bd17bca00 ACCEP

00022bd050--63780704c9e3e243ce94bb3d REJECTED doesn't satisfy the requrements
00022bd050--637806ffbdfa5861cdfe6d50 ACCEPTED good job
00022bd050--637806732e3a4b54f619c4ad ACCEPTED good job
00022bd050--6378070119b2bc661e2ac0f0 ACCEPTED good job
00022bd050--637807018feaf022831bee7d ACCEPTED good job
00022bd050--637807048408eb4b99a821f5 ACCEPTED good job
00022bd050--637807001455e654a92977a5 ACCEPTED good job
00022bd050--6378070c47042e6e2c5dbd4a ACCEPTED good job
00022bd050--63780719ebd2c7014fe359ca ACCEPTED good job
00022bd050--6378071c1455e654a9297828 ACCEPTED good job
00022bd050--637806a68897471c9a93481d REJECTED doesn't satisfy the requrements
00022bd050--6378071ff63f4544852d5d29 ACCEPTED good job
00022bd050--6378073819b2bc661e2ac282 ACCEPTED good job
00022bd050--6378073319b2bc661e2ac26d ACCEPTED good job
00022bd050--6378073519c2d72a00b4f359 REJECTED doesn't satisfy the requrements
00022bd050--63780744d3c878690bd755fe ACCEPTED good job
00022bd050--63780725659b106910926373 ACCEPTED good 

00022bd050--63780990d23d9327ecfdbb6a ACCEPTED good job
00022bd050--63780a08bb79c0321f808c56 ACCEPTED good job
00022bd050--63780996b1ad9403670115ef ACCEPTED good job
00022bd050--63780a2b1d6dd41f3b9dd4ec REJECTED doesn't satisfy the requrements
00022bd050--63780a0abdfa5861cdfe83c3 ACCEPTED good job
00022bd050--63780a17b04cf44326524757 ACCEPTED good job
00022bd050--63780a2fd23d9327ecfdbfaf ACCEPTED good job
00022bd050--63780a14d3a4cf61019c200d ACCEPTED good job
00022bd050--63780a44aabecc6bd17bf718 ACCEPTED good job
00022bd050--63780a3419b2bc661e2ad5fc ACCEPTED good job
00022bd050--63780a4c47042e6e2c5dd58b ACCEPTED good job
00022bd050--63780a6db1ad940367011b80 ACCEPTED good job
00022bd050--63780a7a7545ab02b68d7ed8 ACCEPTED good job
00022bd050--63780a767c35c135550005fb ACCEPTED good job
00022bd050--63780a49b04cf443265248d1 ACCEPTED good job
00022bd050--63780a53389626777f78f3a8 ACCEPTED good job
00022bd050--637809d48d6def0568e3ff0d ACCEPTED good job
00022bd050--63780a7938b93276f2f3d687 ACCEP

00022bd050--63780f076b2d840c16f854fe ACCEPTED good job
00022bd050--63780f058ccdeb4a26e0e84d ACCEPTED good job
00022bd050--63780f2e8ccdeb4a26e0e90c ACCEPTED good job
00022bd050--63780f117c35c1355500219e ACCEPTED good job
00022bd050--63780f0982a8db7f5de4ba88 ACCEPTED good job
00022bd050--63780f298897471c9a93842b REJECTED doesn't satisfy the requrements
00022bd050--63780f178408eb4b99a86142 ACCEPTED good job
00022bd050--63780f2447f33832e652a542 ACCEPTED good job
00022bd050--63780f118cae8c05c827f0a2 ACCEPTED good job
00022bd050--63780f206b2d840c16f85595 ACCEPTED good job
00022bd050--63780ea98feaf022831c2448 ACCEPTED good job
00022bd050--63780e7f9376542ef78a0fee ACCEPTED good job
00022bd050--63780f2ebd8f2378d3540ae3 ACCEPTED good job
00022bd050--63780f3838b93276f2f3f417 ACCEPTED good job
00022bd050--63780f37b88cb76558d0f367 ACCEPTED good job
00022bd050--63780f453903154c8bc0aabf ACCEPTED good job
00022bd050--63780f57bdfa5861cdfea8d7 ACCEPTED good job
00022bd050--63780f4d9376542ef78a1449 ACCEP

...

00022bd050--63788f66b2c0c25adce86379 ACCEPTED good job
00022bd050--63788f5d441f157a2a58e0ff ACCEPTED good job
00022bd050--63788f671d6dd41f3ba1d9a6 ACCEPTED good job
00022bd050--63788f751455e654a92d9f1a ACCEPTED good job
00022bd050--63788f7438b93276f2f75d14 ACCEPTED good job
00022bd050--63788f74b04cf443265a26d9 ACCEPTED good job
00022bd050--63788f7f8ab0c64deb782878 ACCEPTED good job
00022bd050--63788f762e3a4b54f61d9fc7 ACCEPTED good job
00022bd050--63788f7e19c2d72a00b8ebec ACCEPTED good job
00022bd050--63788f857545ab02b690e5ab ACCEPTED good job
00022bd050--63788f84659b1069109783ce ACCEPTED good job
00022bd050--63788f8cff977c7c90534a8e ACCEPTED good job
00022bd050--63788f8dbdfa5861cd022b43 ACCEPTED good job
00022bd050--63788f82bb79c0321f84b5dc ACCEPTED good job
00022bd050--63788f7cbb79c0321f84b5a0 ACCEPTED good job
00022bd050--63788f91441f157a2a58e357 ACCEPTED good job
00022bd050--63788f8b85bacb70b67add54 ACCEPTED good job
00022bd050--63788fa1bdfa5861cd022cb2 REJECTED doesn't satisfy the

00022bd050--637890bc9376542ef78dba90 ACCEPTED good job
00022bd050--637890b7ebd2c7014fe6c51a ACCEPTED good job
00022bd050--637890bc441f157a2a58f061 ACCEPTED good job
00022bd050--637890c182a8db7f5de80a68 ACCEPTED good job
00022bd050--637890bf66569428531f650c ACCEPTED good job
00022bd050--637890beb88cb76558d4f89a ACCEPTED good job
00022bd050--637890c7d23d9327ec012b28 ACCEPTED good job
00022bd050--637890bf9717e27000653608 ACCEPTED good job
00022bd050--637890c3801461320b6c2482 ACCEPTED good job
00022bd050--637890c88ab0c64deb78346d ACCEPTED good job
00022bd050--637890d01455e654a92dabb4 ACCEPTED good job
00022bd050--637890c2c9e3e243ce989264 ACCEPTED good job
00022bd050--637890cde3847d75ff8d0250 ACCEPTED good job
00022bd050--637890d26b2d840c16fb7625 ACCEPTED good job
00022bd050--637890d67adbfb533edf2f32 ACCEPTED good job
00022bd050--637890d7659b106910979104 ACCEPTED good job
00022bd050--637890d247042e6e2c62086c ACCEPTED good job
00022bd050--637890d97545ab02b690f286 ACCEPTED good job
00022bd050

00022bd050--637891cd8feaf022831febb2 ACCEPTED good job
00022bd050--637891ce148cbf49f5691706 ACCEPTED good job
00022bd050--637891d19376542ef78dc534 ACCEPTED good job
00022bd050--637891d49376542ef78dc555 ACCEPTED good job
00022bd050--637891d58feaf022831fec06 ACCEPTED good job
00022bd050--637891d2b04cf443265a6446 ACCEPTED good job
00022bd050--637891d5ff977c7c905367fb ACCEPTED good job
00022bd050--637891da8897471c9a973286 ACCEPTED good job
00022bd050--637891da8ab0c64deb783ed5 ACCEPTED good job
00022bd050--637891da7adbfb533edf380c ACCEPTED good job
00022bd050--637891db838d535336a63bc1 ACCEPTED good job
00022bd050--637891dc66569428531f7166 ACCEPTED good job
00022bd050--637891e219b2bc661e2e12e4 ACCEPTED good job
00022bd050--637891e1389626777f7e86c6 ACCEPTED good job
00022bd050--637891ddc9e3e243ce989d3a ACCEPTED good job
00022bd050--637891e619c2d72a00b90e7c ACCEPTED good job
00022bd050--637891e68ccdeb4a26e42154 ACCEPTED good job
00022bd050--637891f08ccdeb4a26e421bc ACCEPTED good job
00022bd050

00022bd050--637892b47545ab02b6910436 ACCEPTED good job
00022bd050--637892b3b88cb76558d50ffa ACCEPTED good job
00022bd050--637892b6441f157a2a59056d ACCEPTED good job
00022bd050--637892b2148cbf49f5691fbe ACCEPTED good job
00022bd050--637892b8ee7d356a087cffb4 ACCEPTED good job
00022bd050--637892be8408eb4b99ac6869 ACCEPTED good job
00022bd050--637892be38b93276f2f78068 ACCEPTED good job
00022bd050--637892beb58f5e17ab6ef955 ACCEPTED good job
00022bd050--637892bfd3c878690bdb5fa0 ACCEPTED good job
00022bd050--637892c08d6def0568e787dd ACCEPTED good job
00022bd050--637892c056a5a54ced853d44 ACCEPTED good job
00022bd050--637892c0bdfa5861cd024ec0 ACCEPTED good job
00022bd050--637892c1bb79c0321f84dd5d00022bd050--637892c59376542ef78dce13 ACCEPTED good job
 ACCEPTED good job
00022bd050--637892aee3847d75ff8d1581 ACCEPTED good job
00022bd050--637892cd14cd566736a50f79 ACCEPTED good job
00022bd050--637892c69717e27000654a43 REJECTED doesn't satisfy the requrements
00022bd050--637892cf9717e27000654aaa ACCEP

00022bd050--63789460801461320b6c484e ACCEPTED good job
00022bd050--637894699717e27000655927 ACCEPTED good job
00022bd050--6378946419b2bc661e2e2a29 ACCEPTED good job
00022bd050--6378946119c2d72a00b9275e ACCEPTED good job
00022bd050--637894fdbdfa5861cd026658 ACCEPTED good job
00022bd050--63789485ea9fec14e38b0c98 ACCEPTED good job
00022bd050--6378948f5a4d4b776b23387d ACCEPTED good job
00022bd050--6378948d7adbfb533edf5327 ACCEPTED good job
00022bd050--63789507801461320b6c4faa ACCEPTED good job
00022bd050--6378953b9717e2700065616c ACCEPTED good job
00022bd050--637895788897471c9a9777f0 ACCEPTED good job
00022bd050--637895fd47f33832e656fa4f ACCEPTED good job
00022bd050--637896052e3a4b54f61de55b ACCEPTED good job
00022bd050--63789605389626777f7eb0ce ACCEPTED good job
00022bd050--637895d75a4d4b776b2344b2 ACCEPTED good job
00022bd050--637896469717e27000656cd9 ACCEPTED good job
00022bd050--6378963c7c35c135550428b9 REJECTED doesn't satisfy the requrements
00022bd050--63789674bdfa5861cd0274f2 ACCEP

In [440]:
df_agreements.to_csv('batch4.csv', index=False)